# **Pegando os resultados das partidas que aconteceram no dia anterior**
- https://www.placardefutebol.com.br/jogos-de-ontem

# Este código é uma adaptação de um código fornecido em uma das aulas do https://www.youtube.com/@OutspokenMarket

## Bibliotecas

In [1]:
import pandas as pd
import numpy as np 
from datetime import datetime, timedelta


# webscraping
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from time import sleep


# removing warnings
import warnings
warnings.simplefilter('ignore')

## WebScraping da página : https://www.placardefutebol.com.br/jogos-de-ontem

In [2]:
# Inicializa listas para armazenar os dados coletados
data_casa = [] # nome do time da casa
data_fora = [] # time visitante
data_gols_casa = [] # gols do time da casa
data_gols_fora = [] # gols do visitante
data_do_jogo = [] # dia do jogo 
data_status_partida = [] # status da partida , 'encerrado', 'intervalo'


option = webdriver.ChromeOptions()
# Inicializando o navegador Chrome usando o Selenium
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=option)


# Acessando o site 
driver.get(f'https://www.placardefutebol.com.br/jogos-de-ontem')

# espera x segundos para carregar a pagina 
sleep(10)

# maximiza a janela do navegador para garantir que a página foi totalmente carregada
driver.maximize_window()

# tenta fechar pop-up, caso apareca algum 
try:
    driver.find_element(By.XPATH, "//button[@id='onetrust-accept-btn-handler']").click()

except:
    pass

# espera mais x segundos para garantir que os elementos estão disponíveis
sleep(3)


# coletando os dados
clubes_casa = driver.find_elements(By.CLASS_NAME, 'text-right')
clubes_fora = driver.find_elements(By.CLASS_NAME, 'text-left')
gols_casa = driver.find_elements(By.CLASS_NAME, 'justify-content-end')
gols_fora = driver.find_elements(By.CLASS_NAME, 'justify-content-start')
status_partida = driver.find_elements(By.CLASS_NAME, 'status-name')


# data do jogo 
data_hoje = datetime.now()
data_ontem = data_hoje - timedelta(days=1)
# data de ontem formatada
data_ontem_formatada = data_ontem.strftime("%Y-%m-%d")


# Organizando os dados 
for i in range(len(clubes_casa)):
    home, away = clubes_casa, clubes_fora
     # adiciona os dados coletados ás listas 
    data_casa.append(clubes_casa[i].text) # time casa
    data_fora.append(clubes_fora[i].text) # time visitante
    data_gols_casa.append(gols_casa[(i)].text) # gols tima da casa
    data_gols_fora.append(gols_fora[(i)].text) # gols do time visitante
    data_do_jogo.append(data_ontem_formatada)  # data do dia do jogo
    data_status_partida.append(status_partida[i].text) # status da partida 'Encerrado', 'intervalo'

# fechando o nevegador após coletar os dados
driver.close()

## Arrumando os dados 

In [3]:
df = pd.DataFrame({
    'Data': data_do_jogo,
    'Home': data_casa,
    'Away': data_fora,
    'Gols_home': data_gols_casa,
    'Gols_away': data_gols_fora,
    'status': data_status_partida
})


# Arrumando o index
df.set_index('Data', inplace = True)

df.head()

,Home,Away,Gols_home,Gols_away,status
Data,,,,,
2024-10-06,Bayer Leverkusen Feminino,Carl Zeiss Jena Feminino,1,0,ENCERRADO
2024-10-06,Juventud,CA Atenas,1,0,54 MIN
2024-10-06,Hvidovre,Kolding IF,0,0,INTERVALO
2024-10-06,Sokol Saratov,Rotor Volgograd,0,1,56 MIN
2024-10-06,Sport,Ceará,,,HOJE 21:00


In [4]:
df['status'].value_counts()

status
ENCERRADO     82
54 MIN         1
INTERVALO      1
56 MIN         1
HOJE 21:00     1
SUSPENSO       1
Name: count, dtype: int64

## Pegando os jogos com o status 'ENCERRADO'

In [5]:
print(df.shape)

df = df[df['status'] == 'ENCERRADO']

print(df.shape)

(87, 5)
(82, 5)


In [6]:
df.head()

,Home,Away,Gols_home,Gols_away,status
Data,,,,,
2024-10-06,Bayer Leverkusen Feminino,Carl Zeiss Jena Feminino,1,0,ENCERRADO
2024-10-06,Corinthians Feminino,ADIFFEM Feminino,8,0,ENCERRADO
2024-10-06,Olimpia Feminino,Always Ready Feminino,3,0,ENCERRADO
2024-10-06,Boca Juniors Feminino,Libertad Limpeño Feminino,1,0,ENCERRADO
2024-10-06,Colo-Colo Feminino,Santos Feminino,0,1,ENCERRADO
